In [1]:
from skimage import io
from skimage import img_as_uint
from glob import glob
import numpy as np
import yaml
from enhancer import Enhancer
import os.path as path
from lsm_utils import normalize_16bit_images, compute_norm_range

In [5]:
print('Computing normalization range for 16bit data.')
vmin_t, vmax_t, fail_names_t = compute_norm_range('datasets/PA_2081b_part2/target', ext='tif', percentiles=(1, 99.5), sample_r=0.05)
vmin_i, vmax_i, fail_names_i = compute_norm_range('datasets/PA_2081b_part2/input', ext='tif', percentiles=(1, 99.5), sample_r=0.05)
vmin = min(vmin_i, vmin_t)
vmax = max(vmax_i, vmax_t)
if len(fail_names_i + fail_names_t) != 0:
    print('Datasets corrupted: input: {fail_names_i}, target: {fail_neams_t}')

100%|██████████| 1482/1482 [00:31<00:00, 47.10it/s]


In [3]:
vmin, vmax, fail_names = compute_norm_range('datasets/PA_2081b_part2/input', ext='tif', percentiles=(1, 99.5), sample_r=0.05)

100%|██████████| 1482/1482 [00:32<00:00, 45.85it/s]


In [2]:
config = yaml.load(open("model_config.yaml", "r"), Loader=yaml.FullLoader)
config['dataset'] = 'datasets/lsm-pair'

In [9]:
enhancer = Enhancer(config, scale_factor=2, perceptual_loss=True, adversarial_loss=True)

Screening background...


100%|██████████| 18066/18066 [00:44<00:00, 404.07it/s]


In [10]:
import torch
if config['load-weights'] is not None:
    enhancer.backbone.load_state_dict(torch.load(path.join('model_weights', config['load-weights'], 'g.pth')))
    enhancer.discriminator.load_state_dict(torch.load(path.join('model_weights', config['load-weights'], 'd.pth')))

In [7]:
enhancer.train(write_log=True)

>>>> Test Loss - epoch 0: 6.405351320902507, Loss perceptual: 5.75284104598196
>>>> Test Loss - epoch 0: Loss G 0.25110849424412374, Loss D: 0.2504949611529969


  0%|          | 0/139 [00:00<?, ?it/s]

  0%|          | 0/139 [01:46<?, ?it/s]


KeyboardInterrupt: 

In [10]:
enhancer.enhance(sampling=True, sample_rate=0.01)

In [6]:
import torch.nn as nn
import torch
import torchvision.models as models
from torchvision.models.feature_extraction import create_feature_extractor
up = nn.Upsample(size=(299, 299), mode='bilinear')
x = torch.ones((4, 3, 512, 512))

In [10]:
x = up(x).cuda()

In [11]:
x.device

device(type='cuda', index=0)

In [12]:
inception = models.vgg16(pretrained=True).type(torch.float32)
inception.classifier = nn.Identity()
inception

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
resnet = inception
feature_layers = {'layer1' : 'feat1', 'layer2' : 'feat2', 'layer3' : 'feat3', 'layer4' : 'feat4'}
feature_extractor = create_feature_extractor(resnet, feature_layers)
out = feature_extractor(x)

In [17]:
for k, v in out.items():
    print(v)

tensor([[[[8.2564e-01, 5.0183e-02, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.1769e+00, 6.2467e-01, 0.0000e+00,  ..., 0.0000e+00,
           1.8022e-01, 1.4167e+00],
          [2.1974e+00, 1.2527e+00, 0.0000e+00,  ..., 0.0000e+00,
           1.0105e+00, 1.9179e+00],
          ...,
          [2.1743e+00, 8.3548e-01, 0.0000e+00,  ..., 6.1668e-01,
           2.0371e+00, 1.5168e+00],
          [2.8915e+00, 1.6670e+00, 2.3553e-03,  ..., 7.2339e-01,
           2.1676e+00, 1.9244e+00],
          [3.2745e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           3.0392e-01, 0.0000e+00]],

         [[2.3400e+00, 1.2735e+00, 1.4068e+00,  ..., 2.0648e+00,
           1.3560e+00, 6.6883e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           1.2022e-01, 2.5073e-01],
          [1.4032e+00, 1.9179e+00, 1.1980e+00,  ..., 1.6822e+00,
           3.7911e-01, 0.0000e+00],
          ...,
          [2.8113e+00, 2.2366e+00, 2.5502e+00,  ..., 4.4369

In [19]:
out[0].shape

torch.Size([4, 2048])